# Prueba técnica

## Seccion 1

Preguntas:


1. ¿Qué es un dataset desbalanceado y por qué es un problema?
Un dataset desbalanceado es un conjunto de datos donde una clase tiene muchas más muestras que las demás. Esto provoca que el modelo aprenda principalmente los patrones de la clase mayoritaria y no logre identificar bien la clase minoritaria, que suele ser la más importante (como ocurre en fraude).
El principal problema es que el modelo generaliza en exceso, llegando incluso a “ignorar” la clase poco representada, lo que afecta gravemente su capacidad de detección.

2. ¿Qué métrica es más importante para detectar fraude: precisión, recall o accuracy? ¿Por qué?
La métrica más importante para detectar fraude es Recall, porque lo esencial en estos casos es identificar la mayor cantidad posible de fraudes reales.
Perder un fraude (falso negativo) suele ser mucho más costoso que generar una falsa alarma, por lo que maximizar el recall es fundamental.

3. Menciona 3 features útiles para detectar posibles cuentas mula.
- Historial de actividad del usuario.
- Ratio entre transacciones entrantes y salientes.
- Transacciones en las últimas 24 h

4. Explica qué es overfitting.
El overfitting es un problema en los modelos de machine learning donde el modelo aprende no solo los patrones reales del dataset, sino también detalles irrelevantes o ruido del conjunto de entrenamiento.
Es como memorizar respuestas exactas en lugar de comprender el tema: el modelo funciona muy bien con los datos que ya vio, pero fracasa cuando se enfrenta a datos nuevos.

## Seccion 2
Ejercicio práctico:




### Tareas:
#### 2.1. Cargar los datos en Python y realizar limpieza básica.

Cargar los datos (utilizo el account ID como index)

In [13]:
import pandas as pd

df=pd.read_csv('data.csv', index_col='account_id')
df.head()

,age,transactions_last_24h,total_received_last_7d,is_mule
account_id,,,,
1,22,25,15000,1
2,45,3,800,0
3,19,40,22000,1
4,35,5,600,0
5,28,12,4000,0


limpieza basica
- Reviso el tipo de dato de cada columna (`dtype`) y si contiene valores nulos

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 1 to 10
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   age                     10 non-null     int64
 1   transactions_last_24h   10 non-null     int64
 2   total_received_last_7d  10 non-null     int64
 3   is_mule                 10 non-null     int64
dtypes: int64(4)
memory usage: 400.0 bytes


Este df no contiene fechas, sin embargo podriamos haberlas interpretado correctamente en casos de string a date time con el sig comndo

In [15]:
# df['fecha'] = pd.to_datetime(df['fecha'])

en este caso ningun renglón contiene valores nulos en sus columnas, en caso de haber tenido el siguiente comando los habría eliminado

In [16]:
df = df.dropna()

In [17]:
df

,age,transactions_last_24h,total_received_last_7d,is_mule
account_id,,,,
1,22,25,15000,1
2,45,3,800,0
3,19,40,22000,1
4,35,5,600,0
5,28,12,4000,0
6,21,30,18000,1
7,50,2,200,0
8,23,18,9000,0
9,20,33,20000,1


#### 2.2. Crear dos nuevas features basadas en los datos existentes. 

In [18]:
df['ratio'] = df['total_received_last_7d'] / (df['transactions_last_24h'] + 1)

In [19]:
df

,age,transactions_last_24h,total_received_last_7d,is_mule,ratio
account_id,,,,,
1,22,25,15000,1,576.923077
2,45,3,800,0,200.000000
3,19,40,22000,1,536.585366
4,35,5,600,0,100.000000
5,28,12,4000,0,307.692308
6,21,30,18000,1,580.645161
7,50,2,200,0,66.666667
8,23,18,9000,0,473.684211
9,20,33,20000,1,588.235294


In [20]:

df['high_activity_flag']= (df["transactions_last_24h"] > 20).astype(int)

In [21]:
df

,age,transactions_last_24h,total_received_last_7d,is_mule,ratio,high_activity_flag
account_id,,,,,,
1,22,25,15000,1,576.923077,1
2,45,3,800,0,200.000000,0
3,19,40,22000,1,536.585366,1
4,35,5,600,0,100.000000,0
5,28,12,4000,0,307.692308,0
6,21,30,18000,1,580.645161,1
7,50,2,200,0,66.666667,0
8,23,18,9000,0,473.684211,0
9,20,33,20000,1,588.235294,1


Feature: promedio recibido por transacción 

In [22]:
df["avg_received_per_transaction"] = df["total_received_last_7d"] / (df["transactions_last_24h"].replace(0, 1))
df

,age,transactions_last_24h,total_received_last_7d,is_mule,ratio,high_activity_flag,avg_received_per_transaction
account_id,,,,,,,
1,22,25,15000,1,576.923077,1,600.000000
2,45,3,800,0,200.000000,0,266.666667
3,19,40,22000,1,536.585366,1,550.000000
4,35,5,600,0,100.000000,0,120.000000
5,28,12,4000,0,307.692308,0,333.333333
6,21,30,18000,1,580.645161,1,600.000000
7,50,2,200,0,66.666667,0,100.000000
8,23,18,9000,0,473.684211,0,500.000000
9,20,33,20000,1,588.235294,1,606.060606


Feature: bandera de actividad sospechosa (transaccion de mucho dinero en poco tiempo)

In [23]:
df["suspicious_amount_flag"] = (df["total_received_last_7d"] > 10000).astype(int)
df

,age,transactions_last_24h,total_received_last_7d,is_mule,ratio,high_activity_flag,avg_received_per_transaction,suspicious_amount_flag
account_id,,,,,,,,
1,22,25,15000,1,576.923077,1,600.000000,1
2,45,3,800,0,200.000000,0,266.666667,0
3,19,40,22000,1,536.585366,1,550.000000,1
4,35,5,600,0,100.000000,0,120.000000,0
5,28,12,4000,0,307.692308,0,333.333333,0
6,21,30,18000,1,580.645161,1,600.000000,1
7,50,2,200,0,66.666667,0,100.000000,0
8,23,18,9000,0,473.684211,0,500.000000,0
9,20,33,20000,1,588.235294,1,606.060606,1


#### 2.3 Entrenar un modelo de clasificación (Logistic Regression, Random Forest o Decision Tree).

Random Forest: 

In [2]:
from sklearn.ensemble import RandomForestClassifier

#### 2.4. Calcular y mostrar estas métricas:


### 2.5. Explicar brevemente qué variables parecen más importantes para identificar una cuenta mula.